# Task 1

In [ ]:
# Не хватает памяти если выполнять обе задачи параллельно
# Приходится выполнять задачу одна за одной 
# CUBLAS_STATUS_NOT_SUPPORTED https://stackoverflow.com/questions/70689874/pytorch-cannot-handle-complex-tensor-on-gpu-but-works-on-cpu)

In [1]:
!nvidia-smi

Fri Jan 20 13:51:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    56W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [3]:
!python -m pip install paddlepaddle-gpu==2.1.3.post112 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html


In [4]:
!python -c "import paddle; print(paddle.__version__)"

2.1.3


In [5]:
!pip install "paddleocr>=2.0.1" --quiet
!pip install transformers --quiet

In [6]:
import os
import shutil
from PIL import Image
import json
from tqdm.notebook import tqdm

In [ ]:
!git clone https://github.com/Losyash/SROIE-datasetv2.git

In [7]:
sroie_root_dir = './SROIE-datasetv2/'

In [ ]:
# Make dir for test predicted
os.mkdir(os.path.join(sroie_root_dir, 'test_predicted'))
os.mkdir(os.path.join(sroie_root_dir, 'test_predicted/box/'))
os.mkdir(os.path.join(sroie_root_dir, 'test_predicted/entities/'))

In [8]:
sroie_test_img_dir = os.path.join(sroie_root_dir, 'test/img')
sroie_test_pred_ent_dir = os.path.join(sroie_root_dir, 'test_predicted/entities')
sroie_test_pred_box_dir = os.path.join(sroie_root_dir, 'test_predicted/box')

In [9]:
# Boxes
from paddleocr import PaddleOCR

ocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=True)

[2023/01/20 13:51:44] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh=0.5, e2e_pgnet_valid_set

In [10]:
def make_box_file(file_name):

    result = ocr.ocr(f'{sroie_test_img_dir}/{file_name}', cls=True)
    file_name_2 = file_name.split('.')[0] + '-1.txt'

    with open(f'{sroie_test_pred_box_dir}/{file_name_2}', "w") as file:
        for i in range(len(result)):
            res = result[i]

            for line in res:
                data = []

                for j in range(len(line[0])):
                    coords = line[0][j]

                    data.append(str(int(coords[0])))
                    data.append(str(int(coords[1])))

                data.append(str(line[1][0]).upper())
                file.write((',').join(data) + '\n')    

In [11]:
res = ocr.ocr(f'{sroie_test_img_dir}/X00016469670.jpg', cls=True)
print(res)

[2023/01/20 13:51:58] ppocr DEBUG: dt_boxes num : 44, elapse : 2.8344006538391113
[2023/01/20 13:51:58] ppocr DEBUG: cls num  : 44, elapse : 0.0660240650177002
[2023/01/20 13:51:58] ppocr DEBUG: rec_res num  : 44, elapse : 0.08684587478637695
[[[[[103.0, 27.0], [311.0, 27.0], [311.0, 62.0], [103.0, 62.0]], ('tan chay yee', 0.9920644164085388)], [[[140.0, 96.0], [273.0, 96.0], [273.0, 117.0], [140.0, 117.0]], ('***COPY***', 0.8696359395980835)], [[[91.0, 121.0], [325.0, 121.0], [325.0, 138.0], [91.0, 138.0]], ('OJCMARKETING SDN BHD', 0.9541818499565125)], [[[130.0, 144.0], [286.0, 144.0], [286.0, 161.0], [130.0, 161.0]], ('ROCNO:538358-H', 0.9315272569656372)], [[[110.0, 166.0], [303.0, 167.0], [303.0, 184.0], [110.0, 183.0]], ('NO2&4,JALAN BAYU 4', 0.9408490657806396)], [[[126.0, 187.0], [286.0, 188.0], [286.0, 205.0], [126.0, 204.0]], ('BANDAR SERI ALAM,', 0.9438982605934143)], [[[123.0, 209.0], [290.0, 209.0], [290.0, 225.0], [123.0, 225.0]], ('81750 MASAI,JOHOR', 0.9201743006706238)

In [12]:
for file_name in tqdm(os.listdir(sroie_test_img_dir)):
    make_box_file(file_name)
    # make_entities_file(file_name)

print('Done')

  0%|          | 0/347 [00:00<?, ?it/s]

[2023/01/20 13:52:01] ppocr DEBUG: dt_boxes num : 49, elapse : 0.04851055145263672
[2023/01/20 13:52:01] ppocr DEBUG: cls num  : 49, elapse : 0.07019543647766113
[2023/01/20 13:52:01] ppocr DEBUG: rec_res num  : 49, elapse : 0.08797383308410645
[2023/01/20 13:52:01] ppocr DEBUG: dt_boxes num : 63, elapse : 0.062116146087646484
[2023/01/20 13:52:01] ppocr DEBUG: cls num  : 63, elapse : 0.08652257919311523
[2023/01/20 13:52:02] ppocr DEBUG: rec_res num  : 63, elapse : 0.11244082450866699
[2023/01/20 13:52:02] ppocr DEBUG: dt_boxes num : 63, elapse : 0.054110050201416016
[2023/01/20 13:52:02] ppocr DEBUG: cls num  : 63, elapse : 0.08951592445373535
[2023/01/20 13:52:02] ppocr DEBUG: rec_res num  : 63, elapse : 0.1096184253692627
[2023/01/20 13:52:02] ppocr DEBUG: dt_boxes num : 71, elapse : 0.05166268348693848
[2023/01/20 13:52:02] ppocr DEBUG: cls num  : 71, elapse : 0.09660673141479492
[2023/01/20 13:52:02] ppocr DEBUG: rec_res num  : 71, elapse : 0.12061786651611328
[2023/01/20 13:52:0

# Task 2

In [13]:
import torch
# from io import BytesIO
from transformers import VisionEncoderDecoderModel, VisionEncoderDecoderConfig, DonutProcessor

torch.cuda.is_available()

True

In [14]:
processor = DonutProcessor.from_pretrained("unstructuredio/donut-base-sroie")
pretrained_model = VisionEncoderDecoderModel.from_pretrained("unstructuredio/donut-base-sroie")

device = "cuda" if torch.cuda.is_available() else "cpu"
pretrained_model.to(device)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


VisionEncoderDecoderModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0): DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )

In [15]:
def make_entities_file(file_name):
    global pretrained_model, processor

    path_to_test_img = os.path.join(sroie_test_img_dir, file_name)
    image = Image.open(path_to_test_img).convert("RGB")

    task_prompt = f"<s>"
    pixel_values = processor(image, return_tensors="pt").pixel_values
    decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

    # run inference
    outputs = pretrained_model.generate(
        pixel_values.to(device),
        decoder_input_ids=decoder_input_ids.to(device),
        max_length=pretrained_model.decoder.config.max_position_embeddings,
        early_stopping=True,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id,
        use_cache=True,
        num_beams=1,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
    )

    # process output
    prediction = processor.batch_decode(outputs.sequences)[0]
    prediction = processor.token2json(prediction)
    prediction = dict(sorted(prediction.items()))

    for key in ['address', 'company', 'date', 'total']:
        if key not in prediction:
            prediction[key] = ''

    file_name_2 = file_name.split('.')[0] + '-2.txt'

    path_to_test_pred_file = os.path.join(sroie_test_pred_ent_dir, file_name_2)
    with open(path_to_test_pred_file, "w") as fp:
        json.dump(prediction, fp) 


In [16]:
for file_name in tqdm(os.listdir(sroie_test_img_dir)):
    make_entities_file(file_name)

print('Done')

  0%|          | 0/347 [00:00<?, ?it/s]

Done


# Results to cloud

In [19]:
# Copy result to cloud

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
!cp -r /content/SROIE-datasetv2/test_predicted /content/drive/MyDrive/Hackaton2023